In [1]:
# import pyaudio
import wave
import sys
import os
import piplates.POWERplate as POW
import datetime
import logging
import datetime

# DOES NOT RUN ON UBUNTU 20.04 / PYTHON 3.9 BECAUSE I CAN'T GET pyaudiIo TO INSTALL PROPERLY 

On my Ubuntu 20.04 machine I tried to install pyaudio using
```
sudo apt-get install -y python3-pyaudio
```
FAILED

In [2]:
sys.version

'3.7.3 (default, Jan 22 2021, 20:04:44) \n[GCC 8.3.0]'

In [3]:
# These parameters may be set by papermill
# AUDIO_FORMAT = pyaudio.paInt16    # 16-bit resolution
CHANNELS = 1                      # 1 channel
RATE = 44100                      # 44.1 kHz sampling rate
FRAMES_PER_BUFFER = 4096          # 2^12 samples for buffer
RECORD_SECS = 60                  # seconds to record
DEV_INDEX = 1                    # device index found by p.get_device_info_by_index()
# WAV_OUTPUT_FILENAME = 'lunga_up.wav' # name of .wav file

In [4]:
def record1():
    filename = timestamp()
    logging.info(f'recording {filename}')
    os.system(f'arecord -d 60 -r 22000 {timestamp()}.wav')
        
# record1()

In [5]:
def update_github():
    logging.info(f'updating GitHub')
    os.system("git pull")
    os.system("git add .")
    os.system("git commit -m 'commited by record_wav.ipynb'")
    os.system("git push")
# update_github()

In [6]:
def metadata():
    return {'time':str(datetime.datetime.now()), 'volts':POW.getVin(0), 'amps':POW.getIin(0), 'temp':POW.getCPUtemp()}

# metadata()

In [7]:
def poweroff():
    POW.powerOFF(0)

# metadata()

In [8]:
def timestamp():
    '''returns local date and time'''
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f")

# timestamp()

# MAIN

In [ ]:
logging.basicConfig(filename = 'record_wav.log',
                    level=logging.DEBUG, 
                    format='%(asctime)s :: %(levelname)s :: %(message)s')

logging.info(metadata())

record1()
update_github()

logging.info(metadata())
logging.info('FINISHED ----------')

In [ ]:
print('FINISHED')